# Illustrates how to sniff USB traffic:

### Connect:

In [45]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=False)
phy.set_power_source("host")

### Reset the FPGA:

In [46]:
phy.reset_fpga()

### Manually set USB speed:

In [47]:
phy.set_usb_mode_fs()
#phy.set_usb_mode_hs()
#phy.set_usb_mode_ls()

### Tell it how many events to capture:
Maximum is 8192.

In [48]:
phy.set_capture_size(8192)

### Arm:
You should see the blue ARM LED turn on to reflect the armed status. It stays on because no trigger pattern has been programmed yet.

In [63]:
phy.arm(action='capture')

### Program pattern match:
The trigger will occur after this executes. The trigger can be observed on the IO4 pin of the ChipWhisperer connector.

In [64]:
phy.set_pattern(pattern=[0xa5], mask=[0xff]) # use 0x2d for LS target

### Read captured data:

In [66]:
data_times, captured_data, stat_times, captured_stat = phy.read_from_wfifo(entries=8192, verbose=False)

### How much FIFO space went towards timestamp updates?

In [67]:
percent_data = len(captured_data)/8192*100
percent_stat = len(captured_stat)/8192*100
percent_shorttime = sum(phy.short_timestamps)/8192*100
percent_longtime = sum(phy.long_timestamps)/8192*100
percent_longtime_under16 = sum(phy.long_timestamps[0:16])/8192*100
percent_longtime_under32 = sum(phy.long_timestamps[0:32])/8192*100
percent_longtime_under64 = sum(phy.long_timestamps[0:64])/8192*100
percent_longtime_under128 = sum(phy.long_timestamps[0:128])/8192*100
percent_longtime_under512 = sum(phy.long_timestamps[0:512])/8192*100
percent_longtime_over128 = sum(phy.long_timestamps[128:2**14])/8192*100
percent_biglongtimes = sum(phy.long_timestamps[2**14:2**16])/8192*100


print("data: %3.1f%%" % percent_data)
print("stat: %3.1f%%" % percent_stat)
print("short timestamps: %3.1f%%" % percent_shorttime)
print("long timestamps: %3.1f%%" % percent_longtime)
print("long timestamps in range [0,15]: %3.2f%%" % percent_longtime_under16)
print("long timestamps in range [0,31]: %3.2f%%" % percent_longtime_under32)
print("long timestamps in range [0,63]: %3.2f%%" % percent_longtime_under64)
print("long timestamps in range [0,127]: %3.2f%%" % percent_longtime_under128)
print("long timestamps in range [0,511]: %3.2f%%" % percent_longtime_under512)
print("long timestamps in range [128,2**14]: %3.2f%%" % percent_longtime_over128)
print("long timestamps in range [2**14,2**16-1]: %3.2f%%" % percent_biglongtimes)


data: 51.2%
stat: 32.1%
short timestamps: 83.9%
long timestamps: 16.9%
long timestamps in range [0,15]: 0.01%
long timestamps in range [0,31]: 0.01%
long timestamps in range [0,63]: 0.01%
long timestamps in range [0,127]: 2.38%
long timestamps in range [0,511]: 2.38%
long timestamps in range [128,2**14]: 0.79%
long timestamps in range [2**14,2**16-1]: 13.73%


### Short timestamps distribution:

In [72]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(plot_width=800, plot_height=300)

xrange = range(64)
p.line(xrange, phy.short_timestamps, line_color="red")
show(p)

Loading BokehJS ...

### Long timestamps distribution:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure(plot_width=800, plot_height=300)
xrange = 1000
p.line(range(xrange), phy.long_timestamps[0:xrange], line_color="red")
show(p)

### Check the capture FIFO status:
No overflow or underflow events should have occured.

In [71]:
phy.check_fifo_errors()

AssertionError: 

### Interpret captured data:
The pattern match byte which triggered the captured isn't recorded; let's add it back it so that the USB data can be properly interpreted:

In [69]:
data_times.insert(0,0)
captured_data.insert(0,0xa5)

In [70]:
usb = USBInterpreter(False)

import numpy as np
idx = np.where(np.array([data_times[i+1] - data_times[i] for i in range(0, len(data_times)-1)]) > 100)
i = 0
for j in idx[0]:
    chunk = bytes(captured_data[i:(j+1)])
    i = j + 1
    usb.handlePacket(data_times[i], chunk, 0)


[        ]   0.035399 d=  0.035399 [ 93.0 +  3.250] [  3] SETUP: 0.0 
[        ]   0.035408 d=  0.000009 [ 93.0 + 11.833] [ 11] DATA0: 80 06 00 01 00 00 40 00 dd 94 
[        ]   0.035431 d=  0.000024 [ 93.0 + 35.417] [  1] ACK 
[        ]   0.035435 d=  0.000003 [ 93.0 + 38.750] [  3] IN   : 0.0 
[        ]   0.035443 d=  0.000008 [ 93.0 + 47.250] [ 11] DATA1: 12 01 00 02 02 00 00 08 56 5f 
[        ]   0.035470 d=  0.000027 [ 93.0 + 74.000] [  1] ACK 
[        ]   0.035473 d=  0.000003 [ 93.0 + 77.250] [  3] OUT  : 0.0 
[        ]   0.035476 d=  0.000003 [ 93.0 + 80.500] [  3] DATA1: 00 00 
[        ]   0.036570 d=  0.001093 [ 93.0 +1173.667] [  1] ACK 
[        ]   0.071827 d=  0.035257 [149.0 +  3.250] [  3] SETUP: 0.0 
[        ]   0.071836 d=  0.000009 [149.0 + 11.917] [ 11] DATA0: 00 05 0b 00 00 00 00 00 eb 8f 
[        ]   0.071847 d=  0.000011 [149.0 + 23.333] [  1] ACK 
[        ]   0.071850 d=  0.000003 [149.0 + 26.667] [  3] IN   : 0.0 
[        ]   0.071853 d=  0.000003 [1

In [68]:
#Run !pip install crcmod in a cell first

def hd(x):
    return " ".join("%02x" % i for i in x)

class USBInterpreter(object):
    import crcmod
    data_crc = staticmethod(crcmod.mkCrcFun(0x18005))

    def __init__(self, highspeed):
        self.frameno = None
        self.subframe = 0
        self.highspeed = True

        self.last_ts_frame = 0

        self.last_ts_print = 0
        self.last_ts_pkt = 0
        self.ts_base = 0
        self.ts_roll_cyc = 2**24

    def handlePacket(self, ts, buf, flags):
        CRC_BAD = 1
        CRC_GOOD = 2
        CRC_NONE = 3
        crc_check = CRC_NONE
        
        ts_delta_pkt = ts - self.last_ts_pkt
        self.last_ts_pkt = ts

        if ts_delta_pkt < 0:
            self.ts_base += self.ts_roll_cyc

        ts += self.ts_base


        suppress = False

        #msg = "(%s)" % " ".join("%02x" % i for i in buf)
        msg = ""

        if len(buf) != 0:
            pid = buf[0] & 0xF
            if (buf[0] >> 4) ^ 0xF != pid:
                msg += "Err - bad PID of %02x" % pid
            elif pid == 0x5:
                if len(buf) < 3:
                    msg += "RUNT frame"
                else:
                    frameno = buf[1] | (buf[2] << 8) & 0x7
                    if self.frameno == None:
                        self.subframe = None
                    else:
                        if self.subframe == None:
                            if frameno == (self.frameno + 1) & 0xFF:
                                self.subframe = 0 if self.highspeed else None
                        else:
                            self.subframe += 1
                            if self.subframe == 8:
                                if frameno == (self.frameno + 1)&0xFF:
                                    self.subframe = 0
                                else:
                                    msg += "WTF Subframe %d" % self.frameno
                                    self.subframe = None
                            elif self.frameno != frameno:
                                msg += "WTF frameno %d" % self.frameno
                                self.subframe = None
                    
                    self.frameno = frameno
                                
                    self.last_ts_frame = ts
                    suppress = True
                    msg += "Frame %d.%c" % (frameno, '?' if self.subframe == None else "%d" % self.subframe)
            elif pid in [0x3, 0xB, 0x7]:
                n = {3:0, 0xB:1, 0x7:2}[pid]

                msg += "DATA%d: %s" % (n,hd(buf[1:]))

                if len(buf) > 2:
                    calc_check = self.data_crc(buf[1:-2])^0xFFFF 
                    pkt_check = buf[-2] | buf[-1] << 8

                    if calc_check != pkt_check:
                        msg += "\tUnexpected ERR CRC"

            elif pid == 0xF:
                msg += "MDATA: %s" % hd(buf[1:])
            elif pid in [0x01, 0x09, 0x0D, 0x04]:
                if pid == 1:
                    name = "OUT"
                elif pid == 9:
                    name = "IN"
                elif pid == 0xD:
                    name = "SETUP"
                elif pid == 0x04:
                    name = "PING"
                if len(buf) < 3:
                    msg += "RUNT: %s %s" % (name, " ".join("%02x" % i for i in buf))
                else:

                    addr = buf[1] & 0x7F
                    endp = (buf[2] & 0x7) << 1 | buf[1] >> 7

                    msg += "%-5s: %d.%d" % (name, addr, endp)
            elif pid == 2:
                msg += "ACK"
            elif pid == 0xA:
                msg += "NAK"
            elif pid == 0xE:
                msg += "STALL"
            elif pid == 0x6:
                msg += "NYET"
            elif pid == 0xC:
                msg += "PRE-ERR"
                pass
            elif pid == 0x8:
                msg += "SPLIT"
                pass
            else:
                msg += "WUT"

        if not suppress:
            crc_char_d = {
                CRC_BAD: '!',
                CRC_GOOD: 'C',
                CRC_NONE: ' '
            }

            flag_field = "[  %s%s%s%s%s%s]" % (
                'L' if flags & 0x20 else ' ',
                'F' if flags & 0x10 else ' ',
                'T' if flags & 0x08 else ' ',
                'C' if flags & 0x04 else ' ',
                'O' if flags & 0x02 else ' ',
                'E' if flags & 0x01 else ' ')
            delta_subframe = ts - self.last_ts_frame
            delta_print = ts - self.last_ts_print
            self.last_ts_print = ts
            RATE=60.0e6

            subf_print = ''
            frame_print = ''

            if self.frameno != None:
                frame_print = "%3d" % self.frameno

            if self.subframe != None:
                subf_print = ".%d" % self.subframe

            print ("%s %10.6f d=%10.6f [%3s%2s +%7.3f] [%3d] %s " % (
                    flag_field, ts/RATE, (delta_print)/RATE,
                    frame_print, subf_print, delta_subframe/RATE * 1E6,
                    len(buf), msg))

In [ ]:
captured_data[0:8]

In [ ]:
data_times[0:16]